In [1]:
import mindspore as ms
import numpy as np
from mindspore import context
from mindspore import ops
from mindspore import nn
from mindspore import Parameter
from mindspore import Tensor
from mindspore.experimental import optim
from mindspore import context

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

class MutualInformation(nn.Cell):

    def __init__(self,sigma=0.4, num_bins=256, normalize=True):
        super(MutualInformation, self).__init__()

        self.sigma = 2*sigma**2
        self.num_bins = num_bins
        self.normalize = normalize
        self.epsilon = 1e-10

        self.bins = ops.linspace(
            0, num_bins, num_bins).float()

    def marginalPdf(self, values):
        
        residuals = values - ops.unsqueeze(ops.unsqueeze(self.bins, 0), 0)
        kernel_values = ops.exp(-0.5*(residuals / self.sigma).pow(2))

        pdf = ops.mean(kernel_values, axis=1)
        normalization = ops.unsqueeze(ops.sum(pdf, dim=1), 1) + self.epsilon
        pdf = pdf / normalization

        return pdf, kernel_values

    def jointPdf(self, kernel_values1, kernel_values2):
        
        joint_kernel_values = ops.matmul(
            ops.transpose(kernel_values1, (0, 2, 1)), kernel_values2)
        normalization = ops.sum(joint_kernel_values, dim=(
            1, 2)).view(-1, 1, 1) + self.epsilon
        pdf = joint_kernel_values / normalization

        return pdf

    def getMutualInformation(self, input1, input2):
        pdf_x1, kernel_values1 = self.marginalPdf(input1)
        pdf_x2, kernel_values2 = self.marginalPdf(input2)
    
        pdf_x1x2 = self.jointPdf(kernel_values1, kernel_values2)

        H_x1 = -ops.sum(pdf_x1*ops.log2(pdf_x1 + self.epsilon), dim=1)
        H_x2 = -ops.sum(pdf_x2*ops.log2(pdf_x2 + self.epsilon), dim=1)
        H_x1x2 = -ops.sum(pdf_x1x2*ops.log2(pdf_x1x2 +
                            self.epsilon), dim=(1, 2))

        mutual_information = H_x1 + H_x2 - H_x1x2

        if self.normalize:
            mutual_information = 2*mutual_information/(H_x1+H_x2)

        return mutual_information

    def construct(self, input1, input2):
        return self.getMutualInformation(input1, input2)


def KDE(A, Z):
    A_Z = ops.unsqueeze(Z@Z.T, 0)
    A_A = ops.unsqueeze(A, 0)

    input1 = ops.cat([A_A, A_Z])
    input2 = ops.cat([A_Z, A_Z])
    
    MI = MutualInformation(sigma=0.4, num_bins=A_Z.shape[0], normalize=True)

    return MI(input1, input2)[0]



KDE(
    ops.rand((2708, 2708)),
    ops.rand((2708, 16)),
)

[WARNING] ME(18512:139861639210176,MainProcess):2024-03-14-20:19:42.871.835 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.11 and cuda version 11.3.58 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.
[ERROR] CORE(18512,7f341352a4c0,python):2024-03-14-20:19:49.083.087 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_18512/3323798325.py]


Tensor(shape=[], dtype=Float32, value= 0.0182187)